In [1]:
import numpy as np
import feather
from ball_tree import BallTree  # TODO: force python to only look locally for import
from faster_sandwich_filling import multiply_XeeX, CutoffError, GeographyError, get_kernel_fn
from numpy.testing import assert_allclose
from patsy import dmatrices, dmatrix
from core import parse_lat_long, check_parameters
from scipy.sparse import csr_matrix, coo_matrix, diags
from typedefs import ITYPE, DTYPE

In [2]:
%load_ext Cython
%load_ext snakeviz

In [3]:
def conley_cross_section(formula_like, data, lat_long, cutoff, kernel = 'uniform'):
    """Calculate Conley standard errors for a cross section.

    Parameters
    ----------
    formula_like : string or other Patsy formula
        e.g. "my_y_variable = my_X_var1 + my_X_var2"
        See http://patsy.readthedocs.io/en/latest/formulas.html#formulas for
        details on Patsy formulas.
    data : array-like
        Must contain all the variables referenced in the formula.
    lat_long : array_like, or tuple of names of columns in data
        An N-by-2 array of latitudes (in the first column) and longitudes (in
        the second column). Both latitude and longitude should be measured
        in degrees. Valid longitudes are [-90, 90].  Valid latitudes are
        (-180, 180]. The number of rows should be the same as the rows in data.
    cutoff : number
        The maximum distance over which covariance is possible.
        cutoff must be a positive number in the range (0, 20015).
    kernel : string
        The kernel function to weight the distances by. Valid options are:
        'bartlett', 'triangle', 'epanechnikov', 'quartic', 'biweight' and
        'triweight'. (Bartlett is the same as triangle. Quartic is the same as
        biweight.)
    """
    y, X = dmatrices(formula_like, data, eval_env = 1, NA_action = 'raise')
    # TODO: handle cases where people provide weird formulas?

    lat_long = parse_lat_long(lat_long, data)
    # Raise an exception if the data look funky
    nobs = check_parameters(y, X, lat_long, cutoff)

    # I have no idea if this leaf_size is reasonable.  If running out of memory,
    # divide N by a larger number.
    # 40 is the default.
    leaf_size = max(40, nobs // 1000)
    # TODO: consider a more sophisticated way of calculating residuals (e.g. one that
    # allows for fancy fixed effects)
    betahat, _, rank, _ = np.linalg.lstsq(X, y)
    if rank != X.shape[1]:
        raise np.linalg.LinAlgError('X matrix is not full rank!')
    del rank
    residuals = (y - X @ betahat)[0]
    balltree = BallTree(lat_long, metric = 'greatcircle', leaf_size = leaf_size)
    if kernel == 'uniform':
        neighbors = balltree.query_radius(lat_long, r = cutoff)
        filling = multiply_XeeX(neighbors, residuals, X, kernel)
    else:
        neighbors, neighbor_distances = balltree.query_radius(
            lat_long, r = cutoff, return_distance = True)
        filling = multiply_XeeX(neighbors, residuals, X, kernel,
                                distances = neighbor_distances, cutoff = cutoff)
        del neighbor_distances
    del balltree, neighbors, y, residuals

    bread = np.linalg.inv(X.T @ X)
    sandwich = nobs * (bread.T @ filling @ bread)
    se = np.sqrt(np.diag(sandwich)).reshape(-1, 1)
    return se



In [4]:
def test_quakes():
    quakes = feather.read_dataframe('tests/datasets/quakes.feather')
    quakes_lat = quakes['lat'].reshape(-1, 1)
    # Subtract 180 because they've done 0 to 360.  See:
    # https://stackoverflow.com/questions/19879746/why-are-datasetquakes-longtitude-values-above-180
    quakes_long = quakes['long'].reshape(-1, 1) - 180
    quakes_lat_long = np.hstack((quakes_lat, quakes_long))
    cutoff = 100

    # correct_results = conley_unfancy(quakes_y, quakes_X, quakes_lat_long, cutoff)
    correct_results = np.array((108.723235, 19.187791)).reshape(-1, 1)  # faster testing
    fast_results = conley_cross_section("depth ~ mag", quakes,
                                        quakes_lat_long, cutoff)
    assert_allclose(correct_results, fast_results)
# test_quakes()

In [5]:
quakes = feather.read_dataframe('tests/datasets/quakes.feather')
quakes_lat = quakes['lat'].reshape(-1, 1)
quakes_long = quakes['long'].reshape(-1, 1) - 180
quakes_lat_long = np.hstack((quakes_lat, quakes_long))
cutoff = 100

balltree = BallTree(quakes_lat_long, metric = 'greatcircle')
neighbors, distances = balltree.query_radius(quakes_lat_long, r = cutoff, return_distance = True)

    
y, X = dmatrices(data=quakes, formula_like='depth ~ mag')
betahat, _, rank, _ = np.linalg.lstsq(X, y)
if rank != X.shape[1]:
    raise np.linalg.LinAlgError('X matrix is not full rank!')
del rank
residuals = (y - X @ betahat)


In [6]:
def conley_unfancy(y, X, lat_long, cutoff):
    N = y.shape[0]
    k = X.shape[1]
    bread = np.linalg.inv(X.T @ X)  # 'bread' in the sandwich-estimator sense

    # Run OLS to get residuals
    betahat = bread @ X.T @ y  # '@' is matrix multiplication, equivalent to np.dot
    residuals = y - X @ betahat
    meat_matrix = np.zeros((k, k))
    row_of_ones = np.ones((1, N))
    column_of_ones = np.ones((k, 1))
    # every_point_is_a_neighbor_of_every_other = True
    for i in range(N):
        dist = great_circle_one_to_many(lat_long, lat_long[i])

        window = dist <= cutoff
        # if not all(window):
        #     every_point_is_a_neighbor_of_every_other = False
        X_i = X[i, ].reshape(-1, 1)
        residuals_i = residuals[i, ].reshape(-1, 1)

        #         k x 1       1 x n        1 x 1
        XeeXh = (((X_i @ row_of_ones * residuals_i) *
                  (column_of_ones @ (residuals.T * window.T))) @ X)
        #                 k x 1                1 x n            n x k
        meat_matrix += XeeXh
    meat_matrix = meat_matrix / N

    sandwich = N * (bread.T @ meat_matrix @ bread)
    se = np.sqrt(np.diag(sandwich)).reshape(-1, 1)
    return se


In [7]:

# def neighbors_to_sparse_nonuniform(neighbors, distances, kernel, cutoff):
#     nrow = neighbors.shape[0]
#     if distances is None:
#         raise ValueError('You must provide distances if using a non-uniform kernel')
#     if cutoff is None:
#         raise ValueError('You must provide a cutoff if using a non-uniform kernel')
#     if distances.shape[0] != nrow:
#         raise ValueError('neighbors and distances have different numbers of rows')
        
#     neighbors_lil = lil_matrix((nrow, nrow), dtype=np.float_)
#     kernel_fn = get_kernel_fn(kernel)
#     for i, neighbor_list in enumerate(neighbors):
#         assert neighbor_list.shape == distances[i].shape
#         # fun fact: you have to apply the kernel function here; you can't do a sparse matrix 
#         # of distances, since some distances (e.g. distance with self) are zero.
#         neighbors_lil[i, neighbor_list] = kernel_fn(distances[i], cutoff)
#     return neighbors_lil.tocsr()


# def neighbors_to_sparse_uniform(neighbors):
#     nrow = neighbors.shape[0]
#     neighbors_lil = lil_matrix((nrow, nrow), dtype=np.float_)
#     for i, neighbor_list in enumerate(neighbors):
#         neighbors_lil[i, neighbor_list] = 1
#     return neighbors_lil.tocsr()


# def neighbors_to_sparse(neighbors, kernel = 'uniform', distances = None, cutoff = None):
#     if kernel == 'uniform':
#         if cutoff is not None or distances is not None:
#             raise ValueError("this combination of parameters should never be necessary; it's a coding mistake")
#         return neighbors_to_sparse_uniform(neighbors)
#     else:
#         return neighbors_to_sparse_nonuniform(neighbors, distances, kernel, cutoff)

# neighbors_sp = neighbors_to_sparse(neighbors)
# neighbors_sp[1,100]


# nrow = residuals.shape[0]
# # not smart enough to convert an (N, 1) array to a (N,) array, so manually reshape
# resid_diag_matrix = diags(residuals.reshape(-1), offsets=0, shape = (nrow, nrow))
# resid_x_neighbors = neighbors_sp * resid_diag_matrix

# for i in range(residuals.shape[0]):
#     resid_x_neighbors[i, :] *= residuals


#type(neighbors_sp)
#print(resid_x_neighbors, neighbors_sp)
# at_mult = 
# star_mult = X.T * resid_x_neighbors * X

# print(at_mult.shape)
# at_mult, star_mult)
# residuals.reshape(-1).shape

In [8]:


# %%cython
# import numpy as np
# cimport numpy as np
# from typedefs cimport DTYPE_t, ITYPE_t
# from typedefs import DTYPE, ITYPE
# import cython
# from scipy.sparse import coo_matrix, csr_matrix
# np.import_array()


# def neighbors_to_sparse_cython(
#         object[:] neighbors not None, 
#         str kernel not None, 
#         object[:] distances = None, 
#         DTYPE_t cutoff = None):

#     if kernel == 'uniform':
#         if cutoff is not None or distances is not None:
#             err_msg = "this combination of parameters should never be necessary; it's a coding mistake"
#             raise ValueError(err_msg)
#         neighbors_sparse = neighbors_to_sparse_uniform_cython(neighbors)
#     else:
#         if cutoff is None or distances is None:
#             err_msg = "this combination of parameters should never be necessary; it's a coding mistake"
#             raise ValueError(err_msg)
#         neighbors_sparse = neighbors_to_sparse_nonuniform_cython(neighbors, kernel, distances, cutoff)
#     return neighbors_sparse


# @cython.boundscheck(False) # turn off bounds-checking for entire function
# @cython.wraparound(False)  # turn off negative indexing
# cdef neighbors_to_sparse_uniform_cython(object[:] neighbors):
#     cdef ITYPE_t i_neighbor
#     cdef ITYPE_t nrow = len(neighbors)
#     # collect neighbor counts for nnz and the inner loop below
#     cdef ITYPE_t[:] neighbor_counts = np.empty(nrow, dtype=ITYPE)
    
#     for i_neighbor in range(nrow):
#         neighbor_counts[i_neighbor] = len(neighbors[i_neighbor])
#     cdef ITYPE_t nnz = np.sum(neighbor_counts)  # number of non-zeros
    
#     cdef ITYPE_t[:] rows = np.empty(nnz, dtype=ITYPE)
#     cdef ITYPE_t[:] cols = np.empty(nnz, dtype=ITYPE)
#     # for values, use dtype (i.e., float64) because even though it's 
#     # always going to be 1, I want the same type for nonuniform
#     cdef DTYPE_t[:] values = np.ones(nnz, dtype=DTYPE)  
    
#     cdef ITYPE_t[:] neighbors_row
#     cdef ITYPE_t row_idx, neighbor_id
#     cdef ITYPE_t sparse_idx = 0
#     cdef ITYPE_t n_neighbors
#     for row_idx in range(nrow):
#         neighbors_row = neighbors[row_idx]
#         # In case anyone wants to run this in parallel, release the GIL
#         with nogil:
#             for neighbor_id in range(neighbor_counts[row_idx]):
#                 rows[sparse_idx] = row_idx
#                 cols[sparse_idx] = neighbors_row[neighbor_id]
#                 sparse_idx += 1
#     # Finally, we construct a regular SciPy sparse matrix:
#     return coo_matrix((values, (rows, cols)), shape=(nrow, nrow)).tocsr()




# cdef DTYPE_t bartlett_once(DTYPE_t dist, DTYPE_t cutoff) nogil:
#     """Weight distances by the Bartlett (triangular) kernel.

#     Important: The function _does not_ check weather the distance is outside the
#     cutoff.  You should do this elsewhere.

#     Input: A memoryview of distances (float64) and a cutoff (float64).
#     Output: A memoryview of weights (float64) in the range [0, 1].
#     """
#     cdef DTYPE_t weight = 1 - (dist / cutoff)
#     return weight


# cdef DTYPE_t epanechnikov_once(DTYPE_t dist, DTYPE_t cutoff) nogil:
#     """Weight distances by the Epanechnikov kernel.

#     Important: The function _does not_ check weather the distance is outside the
#     cutoff.  You should do this elsewhere.

#     Input: A distance (float64) and a cutoff (float64).
#     Output: A weight (float64) in the range [0, 1].
#     """
#     cdef DTYPE_t weight = (0.75 / cutoff) * (1 - (1 / (cutoff**2)) * dist**2)
#     return weight


# # cdef kernel_dict = {
# #     'bartlett': bartlett_once,
# #     'Bartlett': bartlett_once,
# #     'triangle': bartlett_once,
# #     'Epanechnikov': epanechnikov_once,
# #     'epanechnikov': epanechnikov_once,
# #     }

# @cython.boundscheck(False) # turn off bounds-checking for entire function
# @cython.wraparound(False)  # turn off negative indexing
# cpdef neighbors_to_sparse_nonuniform_cython(
#         object[:] neighbors, 
#         str kernel, 
#         object[:] distances, 
#         DTYPE_t cutoff):
#     assert cutoff is not None
#     kernel_fn = get_kernel_fn(kernel)
# #     assert callable(kernel_fn)
#     cdef ITYPE_t i_neighbor
#     cdef ITYPE_t nrow = len(neighbors)
#     # collect neighbor counts for nnz and the inner loop below
#     cdef ITYPE_t[:] neighbor_counts = np.empty(nrow, dtype=ITYPE)
    
#     for i_neighbor in range(nrow):
#         neighbor_counts[i_neighbor] = len(neighbors[i_neighbor])
#     cdef ITYPE_t nnz = np.sum(neighbor_counts)  # number of non-zeros
    
#     cdef ITYPE_t[:] rows = np.empty(nnz, dtype=ITYPE)
#     cdef ITYPE_t[:] cols = np.empty(nnz, dtype=ITYPE)
#     # for values, use dtype (i.e., float64) because even though it's 
#     # always going to be 1, I want the same type for nonuniform
#     cdef DTYPE_t[:] values = np.empty(nnz, dtype=DTYPE)  
    
#     cdef ITYPE_t[:] neighbors_row
# #     cdef DTYPE_t[:] distances_row
#     cdef DTYPE_t[:] value_row
#     cdef ITYPE_t row_idx, neighbor_idx
#     cdef ITYPE_t sparse_idx = 0
#     cdef ITYPE_t n_neighbors
#     for row_idx in range(nrow):
#         n_neighbors = neighbor_counts[row_idx]  # count of neighbors in this row
#         neighbors_row = neighbors[row_idx]  # neighbor IDs for this row
# #         distances_row = distances[row_idx]
#         # call vectorized kernel fn and paste values in
#         value[sparse_idx: (sparse_idx + n_neighbors + 1)] = kernel_fn(distances[row_idx], cutoff)
#         # In case anyone wants to run this in parallel, release the GIL
#         with nogil:
#             for neighbor_idx in range(n_neighbors):
#                 rows[sparse_idx] = row_idx
#                 cols[sparse_idx] = neighbors_row[neighbor_idx]
# #                 values[sparse_idx] = kernel_fn(distances_row[neighbor_id], cutoff)
#                 sparse_idx += 1
#     # Finally, we construct a regular SciPy sparse matrix:
#     return coo_matrix((values, (rows, cols)), shape=(nrow, nrow)).tocsr()




In [9]:
# %timeit neighbors_to_sparse_uniform_cython(neighbors)
# %timeit neighbors_to_sparse_nonuniform_cython(neighbors, 'epanechnikov', distances, cutoff)
# %timeit neighbors_to_sparse_nonuniform_cython(neighbors, 'bartlett', distances, cutoff)

In [14]:

def neighbors_to_sparse_uniform_old(neighbors):
    nrow = neighbors.shape[0]
    neighbors_lil = lil_matrix((nrow, nrow), dtype=np.float_)
    for i, neighbor_list in enumerate(neighbors):
        neighbors_lil[i, neighbor_list] = 1
    return neighbors_lil.tocsr()
#%timeit neighbors_to_sparse_uniform(neighbors)


def neighbors_to_sparse_uniform(neighbors):
    nrow = len(neighbors)
    nnz = 0  # number of non-zeros
    for i_neighbor in range(nrow):
        nnz += len(neighbors[i_neighbor])
    rows = np.empty(nnz, dtype=ITYPE)
    cols = np.empty(nnz, dtype=ITYPE)
    vals = np.ones(nnz,  dtype=DTYPE)  
    sparse_idx = 0
    for row_idx in range(nrow):
        neighbors_row = neighbors[row_idx]
        n_neighbors = len(neighbors_row)
        end_idx = sparse_idx + n_neighbors
        rows[sparse_idx: end_idx] = row_idx
        cols[sparse_idx: end_idx] = neighbors_row
        sparse_idx += n_neighbors
    assert rows.shape[0] == cols.shape[0] == nnz

    # Finally, we construct a regular SciPy sparse matrix:
    return coo_matrix((vals, (rows, cols)), shape=(nrow, nrow)).tocsr()


def neighbors_to_sparse_nonuniform(neighbors, kernel, distances, cutoff):
    kernel_fn = get_kernel_fn(kernel)
    nrow = len(neighbors)
    
    nnz = 0  # number of non-zeros
    for i_neighbor in range(nrow):
        nnz += len(neighbors[i_neighbor])
    rows = np.empty(nnz, dtype=ITYPE)
    cols = np.empty(nnz, dtype=ITYPE)
    vals = np.empty(nnz, dtype=DTYPE)  
    sparse_idx = 0
    for row_idx in range(nrow):
        neighbors_row = neighbors[row_idx]
        n_neighbors = len(neighbors_row)
        end_idx = sparse_idx + n_neighbors
        rows[sparse_idx: end_idx] = row_idx
        cols[sparse_idx: end_idx] = neighbors_row
        vals[sparse_idx: end_idx] = kernel_fn(distances[row_idx], cutoff)
        sparse_idx += n_neighbors
    assert rows.shape[0] == cols.shape[0] == vals.shape[0] == nnz

    # Finally, we construct a regular SciPy sparse matrix:
    return coo_matrix((vals, (rows, cols)), shape=(nrow, nrow)).tocsr()


def neighbors_to_sparse(neighbors, kernel = 'uniform', distances = None, cutoff = None):
    if kernel == 'uniform':
        if cutoff is not None or distances is not None:
            err_msg = "this combination of parameters should never be necessary; it's a coding mistake"
            raise ValueError(err_msg)
        neighbors_sparse = neighbors_to_sparse_uniform(neighbors)
    else:
        if cutoff is None or distances is None:
            err_msg = "this combination of parameters should never be necessary; it's a coding mistake"
            raise ValueError(err_msg)
        if len(neighbors) != len(distances):
            err_msg = "Number of neighbors and distances don't match."
            raise ValueError(err_msg)
        neighbors_sparse = neighbors_to_sparse_nonuniform(neighbors, kernel, distances, cutoff)
    return neighbors_sparse

%timeit neighbors_to_sparse_uniform(neighbors)
%timeit neighbors_to_sparse_nonuniform(neighbors, 'bartlett', distances, cutoff)

100 loops, best of 3: 5.7 ms per loop
100 loops, best of 3: 9.96 ms per loop


In [15]:

def test_neighbors_to_sparse(neighbors, kernel, distances, cutoff):
    from scipy.sparse import find
    neighbors_sparse_nodistance = neighbors_to_sparse(neighbors)
    neighbors_sparse_withdistance = neighbors_to_sparse(neighbors, kernel, distances, cutoff)
    kernel_fn = get_kernel_fn(kernel)
    for i in range(neighbors.shape[0]):
        neighbors_row_argsort = np.argsort(neighbors[i])  # get the indexes that will sort the row
        neighbors_row_sorted = neighbors[i][neighbors_row_argsort]
        
        distance_row_sorted = kernel_fn(distances[i][neighbors_row_argsort], cutoff)
        
        # test that the neighbor indexes are the same
        np.testing.assert_equal(find(neighbors_sparse_nodistance.getrow(i))[1], neighbors_row_sorted)
        np.testing.assert_equal(find(neighbors_sparse_withdistance.getrow(i))[1], neighbors_row_sorted)
        # test that the distance weight values are the same
        np.testing.assert_equal(find(neighbors_sparse_withdistance.getrow(i))[2], distance_row_sorted)
# just picked one kernel for testing
test_neighbors_to_sparse(neighbors, 'epanechnikov', distances, cutoff)


In [19]:

def test_mult(X, residuals, neighbors, kernel, distances = None, cutoff = None):
    N = X.shape[0]
    k = X.shape[1]
    
    
    meat_matrix = np.zeros((k, k))
    row_of_ones = np.ones((1, N))
    column_of_ones = np.ones((k, 1))
    # neighbors_to_sparse will get the uniform or the kernel-ized version, as necessary
    neighbors_sp = neighbors_to_sparse(neighbors, kernel, distances, cutoff )
    
    neighbors_dense = neighbors_sp.toarray()
    for i in range(N):
        window = neighbors_dense[i, :]
        X_i = X[i, ].reshape(-1, 1)
        residuals_i = residuals[i, ].reshape(-1, 1)
        
        #         k x 1       1 x n        1 x 1
        XeeXh = (((X_i @ row_of_ones * residuals_i) *
                  (column_of_ones @ (residuals.T * window.T))) @ X)
        #                 k x 1                1 x n            n x k
        meat_matrix += XeeXh
    correct = meat_matrix / N
    
    # I want element-wise multiplication of the residuals vector by the neighbors weights matrix.
    # Sparse matrices don't have element-wise multiplication, but it's equivalent to cast 
    # the residuals as a sparse diagonal matrix, then do matrix multiplication.
    # The diags function isn't smart enough to convert an (N, 1) array to a (N,) array, so manually reshape.
    # Then, unlike numpy arrays, with sparse matrices, '*' means matrix multiplication, NOT element-wise.
    resid_diag_matrix = diags(residuals.reshape(-1), offsets = 0, shape = (N, N))    
    resid_x_neighbors = resid_diag_matrix * neighbors_sp * resid_diag_matrix
    
    proposed = (X.T @ resid_x_neighbors @ X) / N
    np.testing.assert_allclose(correct, proposed)
    
    bread = np.linalg.inv(X.T @ X)
    correct_sandwich = N * (bread.T @ correct @ bread)
    correct_se = np.sqrt(np.diag(correct_sandwich)).reshape(-1, 1)
    proposed_sandwich = N * (bread.T @ proposed @ bread)
    proposed_se = np.sqrt(np.diag(proposed_sandwich)).reshape(-1, 1)
    np.testing.assert_allclose(correct_se, proposed_se)
    
    
test_mult(X, residuals, neighbors, 'uniform')
test_mult(X, residuals, neighbors, 'epanechnikov', distances, cutoff)
test_mult(X, residuals, neighbors, 'bartlett', distances, cutoff)

